# Generate Map

> Class that generates the data map

In [ ]:
#| default_exp app

In [ ]:
#| export
from nbdev.showdoc import *
try:
    from SolomonIslandsDataMap.app_data import stored_data, sidebar
except: 
    from app_data import stored_data, sidebar

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # Unless this is used graphs will not be dynamic?
import numpy as np
from fastcore.test import *
from dash import page_container, Dash, dcc, Output, Input, State, html, Patch, page_registry, ctx  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
from dash_bootstrap_templates import load_figure_template
from git import Repo
import json


## Setup App and Server

Setup the app and global variables

In [ ]:
#| export

# Find the absoulte path to the pages folder
repo = Repo('.', search_parent_directories=True)
fp = str(repo.working_tree_dir) + "/SolomonIslandsDataMap/pages/"

try:
    app = Dash(__name__, external_stylesheets=[dbc.themes.MINTY,  dbc.icons.BOOTSTRAP], use_pages=True, pages_folder = fp)
except:
    # When running in a notebook, the below trick should get the notebook to still execute
    import __main__ as main
    main.__file__ = "main_file"
    app = Dash(__name__, external_stylesheets=[dbc.themes.MINTY,  dbc.icons.BOOTSTRAP], use_pages=True, pages_folder = fp)
server = app.server
load_figure_template("minty")


In [ ]:
print(fp)

/home/tom/git/SolomonIslandsDataMap/SolomonIslandsDataMap/pages/


Create a navbar

In [ ]:
#| export
pages = {}
for page in page_registry.values():
    pages[page["title"]] = page["relative_path"]

left_nav = dbc.Nav(
    [
        dbc.NavItem(dbc.NavLink('2009 Census', href=pages['Census Map'], active=True, id='censusNav')),
        dbc.NavItem(dbc.NavLink('Election Map', href=pages['Election Map'], active=False, id='electionNav')),
        dbc.NavItem(dbc.NavLink('Population Projections', href=pages['Population Projections'], active=False, id='popNav')),
        dbc.NavItem(dbc.NavLink('Data Table', href=pages['Data Table'], active=False, id='tableNav')),
        dbc.DropdownMenu(
            children=[
                dbc.DropdownMenuItem('2009 Census', href=pages['Census Map']),
                dbc.DropdownMenuItem('Election Map', href=pages['Election Map']),
                dbc.DropdownMenuItem('Population Projections', href=pages['Population Projections']),
                dbc.DropdownMenuItem('Data Table', href=pages['Data Table']),
            ],
            nav=True,
            in_navbar=True,
            label="Select Page",
        ),
    ],
    className="me-auto",   # pushes right nav to right
    navbar=True
)

right_nav = dbc.Nav(
    [
        dbc.NavItem(
            dbc.NavLink(
                html.I(className="bi bi-github"),
                href="https://github.com/Gippers",
                target="_blank"
            )
        ),
        dbc.NavItem(
            dbc.NavLink(
                html.I(className="bi bi-globe"),
                href="https://www.thomasmartin.id.au",
                target="_blank"
            )
        ),
    ],
    navbar=True,
    className="ms-auto"  # RIGHT SIDE
)

navbar = dbc.Navbar(
    dbc.Container(
        [
            dbc.NavbarBrand("Solomon Islands Data Explorer", href="#"),
            left_nav,
            right_nav,
        ],
        fluid=True
    ),
    color="primary",
    #dark=True,
    className="navbar navbar-expand-lg bg-primary"
)


Import the sidebar

## Create the Layout

In [ ]:
#| export
app.layout = dbc.Container([
                dbc.Row([
                    navbar
                ]),
                dbc.Row(
                [dbc.Col(sidebar, width = 2),
                dbc.Col([
                    page_container, ], width = 10),
                ], justify = 'center'),
                #stored_data, # removing stored data for now, note necessary unless using for clientside callbacks                                    
                ], fluid = True)

## Run the dash app

In [ ]:
#| export
#| server
# Run app
if __name__=='__main__':
    try:
        app.run_server(debug=True, port=9999) # Random int mitigates port collision
    except:
        print("Cannot run server here")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

Func: update_geography
Func: update_map
Updating map for census
Func: map_click
Func: bar_click
Func: map_selections
Func: update_bargraph
Func: update_geography
Func: update_geography
Func: update_geography
Func: update_geography
Func: update_measure
